In [74]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import nltk

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('treebank')
from nltk.chunk import tree2conlltags
from nltk.corpus import names
from nltk import word_tokenize
import re
import csv 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import en_core_web_sm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk import pos_tag
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import wordnet as wn
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import seaborn as sns; sns.set_theme()
from sklearn import preprocessing
from sklearn.metrics import classification_report

# loading test and training data

In [75]:
df=pd.read_csv(r"/kaggle/input/nlp-getting-started/train.csv",encoding='latin-1')

In [76]:
df_test=pd.read_csv(r"/kaggle/input/nlp-getting-started/test.csv",encoding='latin-1')

# drop columns we don't use

In [77]:
df=df.drop(['id','location','keyword'],axis='columns')
df.replace(np.nan, 'Miss', inplace=True)

In [78]:
df_test=df_test.drop(['id','location','keyword'],axis='columns')
df_test.replace(np.nan, 'Miss', inplace=True)

# Applying pre-processing techniques

In [80]:
def pre_processing(tweets,boolean):
    dataCaracteres = []
    for linha in tweets:
        linha[0]=re.sub(r'^https?:\/\/.*[\r\n]*', '', linha[0], flags=re.MULTILINE)
        linha[0]=re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ:; ]', '', linha[0])
        linha[0]=linha[0].replace("\n","")
        linha[0] = re.sub(r'@\S+','',linha[0])
        linha[0] = re.sub(r'#\S+','',linha[0]) 
        linha[0] = re.sub(r'https?://\S+|www\.\S+|http?://\S+','',linha[0]) 
        linha[0] = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', linha[0])  
        linha[0] = re.sub(r'[0-9]', '', linha[0])
        linha[0] = re.sub("["
                            u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',linha[0])
        
        dataCaracteres.append(linha)
    if boolean:
        base =pd.DataFrame(dataCaracteres,columns=['text','target'])
    else:
        base =pd.DataFrame(dataCaracteres,columns=['text'])
    base = base.apply(lambda x: x.astype(str).str.lower()).apply(lambda x:x.astype(str).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))


    base['text']= [word_tokenize(entry) for entry in base['text']]
    base['text'].dropna(inplace=True)
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV

    for index,entry in enumerate(base['text']):
        Final_words = []
        word_Lemmatized = WordNetLemmatizer()
        for word, tag in pos_tag(entry):
            if word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
        base.loc[index,'text'] = str(Final_words)
        
    return base

In [81]:
tweets=df.to_numpy()
tweets_test = df_test.to_numpy()

In [82]:

df_1 = pre_processing(tweets,True)
df_1.to_csv('preprocessing.csv', index=False)
df_1.head()

## this step is not necessary. I used to not apply pre-processing techniques to each execution

In [83]:

df_test = pre_processing(tweets_test,False)
df_test.to_csv('preprocessing_test.csv', index=False)
df_test.head()

In [84]:
df_2=pd.read_csv(r"preprocessing.csv",encoding='latin-1')
df_2_test=pd.read_csv(r"preprocessing_test.csv",encoding='latin-1')

# I tested some techniques to balance the classes, but it didn't get a good performance.

In [85]:
from imblearn.under_sampling import RandomUnderSampler
def under_sampler(X,Y):
    rus = RandomUnderSampler()
    return rus.fit_resample(X, Y)

# separating into test and training

In [86]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split (df_2['text'], df_2['target'], test_size = 0.20)

# applying tf-idf

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
Encoder = LabelEncoder () 
Train_Y = Encoder.fit_transform (Train_Y) 
Test_Y = Encoder.fit_transform (Test_Y)
    
Tfidf_vect = TfidfVectorizer (max_features=10000) 
Tfidf_vect.fit(df_2['text'])
Train_X_Tfidf = Tfidf_vect.transform (Train_X) 
Test_X_Tfidf = Tfidf_vect.transform (Test_X)

# generating our training model and plotting the respective confusion matrix

In [88]:
SVM = svm.SVC(class_weight={1: 1.427422806481},C=0.5, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)


In [89]:
cv = StratifiedKFold(n_splits = 5, shuffle = True)
y_pred = cross_val_predict(SVM, Test_X_Tfidf, Test_Y, cv = cv)
fig, ax = plt.subplots()
sns.heatmap(confusion_matrix(Test_Y, y_pred), annot=True, 
            ax=ax, fmt='d', cmap='Reds')
ax.set_title("Matriz de Confusão", fontsize=18)
ax.set_ylabel("True label")
ax.set_xlabel("Predicted Label")
# relatório do modelo
print('Relatório de classificação:\n', classification_report(Test_Y, y_pred, digits=4))

![](https://i.gyazo.com/f7b31510dd705fe4b10f2766614cc6e3.png)

# applying k-folds to find average accuracy

In [90]:
from sklearn.model_selection import ShuffleSplit
X=df_2['text']
y=df_2['target']

base=pd.DataFrame()

#modelo
modelo = svm.SVC(class_weight={1: 1.427422806481},C=0.5, kernel='linear', degree=3, gamma='auto')

# Vamos criar 10 separações, mantendo 20% de teste em cada uma
rs = ShuffleSplit(n_splits=5, test_size=.20)

# List para armazernamos as acurácias de cada modelo
acuracias = []
# Agora iteramos por cada separação e treinamos um modelo
# O método split retorna índices para o vetor passado como parametro
# O .loc é a maneira de indexar tabelas no pandas a partir de índices
predicoes_array=[]
count=0
for train_index, test_index in rs.split(X):
      
    Encoder = LabelEncoder () 
    train_y = Encoder.fit_transform (y.loc[train_index]) 
    test_y = Encoder.fit_transform (y.loc[test_index]) 
    Tfidf_vect = TfidfVectorizer (max_features=10000) 
    Tfidf_vect.fit (df_2['text'])
    train_x = Tfidf_vect.transform (X.loc[train_index]) 
    test_x= Tfidf_vect.transform (X.loc[test_index])  
  #print(train_x.sample(n=5))
  #auxX, auxY = under_sampler(train_x, train_y)
    auxX=train_x
    auxY=train_y
    modelo_treinado = modelo.fit(auxX,auxY)
    predicoes = modelo_treinado.predict(test_x)
    base[count] = predicoes
    acuracias.append(accuracy_score(test_y, predicoes))
    count=count+1
print(acuracias)
print("Média das acurácias: ", sum(acuracias) / len(acuracias))
#Uma abordagem seria estratificar a separação entre treino/teste. A estratificação procura garantir que a distribuição dos dados é semelhante entre os grupos. Ela pode ser feita tanto para atributos como para rótulos. 


# generating our final model and saving the submission document

In [91]:
train = df_2.drop('target',axis='columns')

Encoder = LabelEncoder () 
Train_Y = Encoder.fit_transform (df_2['target']) 
 
Tfidf_vect = TfidfVectorizer (max_features=10000) 
Tfidf_vect.fit(df_2['text'])
Train_X_Tfidf = Tfidf_vect.transform (train['text']) 
Test_X_Tfidf = Tfidf_vect.transform (df_2_test['text'])

In [92]:
modelo_final = svm.SVC(class_weight={1: 1.327422806481},C=0.5, kernel='linear', degree=3, gamma='auto')
modelo_final.fit(Train_X_Tfidf,Train_Y)

In [93]:
predicoes=SVM.predict(Test_X_Tfidf)

In [94]:
submission = pd.read_csv(r'/kaggle/input/nlp-getting-started/sample_submission.csv')

In [95]:
submission['target'] = predicoes

In [96]:
submission

In [97]:
submission.to_csv('./submission.csv', index = False)
print('./submission.csv')